In [11]:
import os 
import re
import transliterate

import pandas as pd
import networkx as nx

In [3]:
DATA_DIR = "/Users/n.anokhin/Documents/data/recsys-overview/"

In [4]:
nodes = pd.read_parquet(os.path.join(DATA_DIR, "nodes.parquet"))

nodes.head(3)

,name,genre,artistId
0,DJ LAVRUSHKIN & EDDIE G,Не указан,122882853519956
1,Edenbridge,Power Metal,122891263398557
2,Sergio,Не указан,26674525905474


In [5]:
edges = pd.read_parquet(os.path.join(DATA_DIR, "edges.parquet"))

edges.head(3)

,sourceId,destinationId,weight
0,7425824682567,7433490145954,0.571782
1,7430282658492,122882867139153,0.454767
2,7458507653680,122882938397254,0.460047


In [12]:
def normalize(string):
    transliterated = transliterate.translit(string, "ru", reversed=True)
    return re.sub(r'[^a-zA-Z0-9]', '', transliterated)
    

graph = nx.Graph()

for _, node in nodes.iterrows():
    graph.add_node(
        node["artistId"], 
        **{
            "name": normalize(node["name"]), 
            "genre": normalize(node["genre"]),
        },
    )
    
graph.add_edges_from([(edge.sourceId, edge.destinationId) for edge in edges.itertuples()])

In [13]:
nx.write_gexf(graph, os.path.join(DATA_DIR, "ok-music.gexf"))